## Join the Duet Server the Data Owner 1 connected to

In [4]:
import syft as sy
duet1 = sy.join_duet(loopback=True,network_url="http://localhost:5000/")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://localhost:5000/
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


/home/holly/anaconda3/envs/pysyft/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/holly/anaconda3/envs/pysyft/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [5]:
duet2 = sy.join_duet(loopback=True,network_url="http://localhost:5000/")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://localhost:5000/
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [6]:
duet1.store.pandas

ID                    Tags  \
0  <UID: 4ba64b83a08f4192801bb372dfb905af>    [breast_cancer-data]   
1  <UID: b6c148b8531e4190ba778bb16b9db0b0>  [breast_cancer-target]   

                                  Description             object_type  
0  这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集其中200条样本，共30维的特征数据  <class 'torch.Tensor'>  
1                      这是数据标签，1代表良性肿瘤，0代表恶性肿瘤  <class 'torch.Tensor'>

In [7]:
duet2.store.pandas

ID                    Tags  \
0  <UID: 1fc48184457144e1ad8627f80c1217cf>    [breast_cancer-data]   
1  <UID: 0b4d398b2db24db6beabaae1e5c99666>  [breast_cancer-target]   

                                 Description             object_type  
0  这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集总296条样本，共30维的特征数据  <class 'torch.Tensor'>  
1                     这是数据标签，1代表良性肿瘤，0代表恶性肿瘤  <class 'torch.Tensor'>

In [8]:
data_ptr1 = duet1.store[0]
target_ptr1 = duet1.store[1]
data_ptr2 = duet2.store[0]
target_ptr2 = duet2.store[1]

In [9]:
print(data_ptr1)
print(target_ptr1)
print(data_ptr2)
print(target_ptr2)

In [10]:
print(duet1.store.pandas["Description"][0])
print()
print(duet1.store.pandas["Description"][1])
print(duet2.store.pandas["Description"][0])
print()
print(duet2.store.pandas["Description"][1])

这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集其中200条样本，共30维的特征数据

这是数据标签，1代表良性肿瘤，0代表恶性肿瘤
这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集总296条样本，共30维的特征数据

这是数据标签，1代表良性肿瘤，0代表恶性肿瘤


### Create Base Model

In [11]:
import torch

In [12]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.fc1 = self.torch_ref.nn.Linear(30, 10)
        self.fc2 = self.torch_ref.nn.Linear(10, 1)
        self.sigmoid=self.torch_ref.nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [13]:
local_model = SyNet(torch)

将模型发动给dataowner1和dataowner2

In [14]:
remote_model1 = local_model.send(duet1)

In [15]:
remote_model2 = local_model.send(duet2)

远程的torch

In [16]:
remote_torch1 = duet1.torch

In [17]:
remote_torch2 = duet2.torch

设置优化器

In [18]:
params1 = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params1, lr=0.01)
params2 = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params2, lr=0.01)

In [22]:
async def train(ownername,iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []
    #损失函数
    cost=torch_ref.nn.BCELoss()
    for i in range(iterations):
        
        #预测结果
        output = model(data_ptr)
        
        #计算损失值
        loss=cost(output,target_ptr)
        
        #在反向传播前先把梯度清零
        optim.zero_grad()
        
        #反向传播,计算各参数对于损失loss的梯度
        loss.backward()
        
        #根据刚刚反向传播得到的梯度更新模型参数
        optim.step()        

        loss_item = loss.item()
        
        loss_value = loss_item.get(
            reason="To evaluate training progress", request_block=True, timeout_secs=5,delete_obj=False)

        if i % 100 == 0:
            print(ownername,"--Epoch", i, "loss", loss_value)
        ##返回损失
        losses.append(loss_value)
        
    #返回模型
    latest_model=model.get(
        request_block=True,
        reason="To run test and inference locally",
        timeout_secs=5,
    )

    return 0

设置迭代次数

In [23]:
iteration = 400

In [24]:
import asyncio
results = await asyncio.gather(
        *[
            train("dataowner1",iteration, remote_model1, remote_torch1, optim1, data_ptr1, target_ptr1),
            train("dataowner2",iteration, remote_model2, remote_torch2, optim2, data_ptr2, target_ptr2)
        ]
    )

dataowner2 --Epoch 0 loss 0.7461003065109253
dataowner2 --Epoch 100 loss 0.5986180305480957
dataowner2 --Epoch 200 loss 0.423175573348999
dataowner2 --Epoch 300 loss 0.2617560029029846
dataowner1 --Epoch 0 loss 0.5799753665924072
dataowner1 --Epoch 100 loss 0.3977128267288208
dataowner1 --Epoch 200 loss 0.2465762197971344
dataowner1 --Epoch 300 loss 0.1673269271850586


### 获取DataOwner1和DataOwner2模型参数

In [25]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('fc1.weight', tensor([[ 0.1978,  1.0762, -0.5464, -2.5852,  2.4051, -0.6892, -1.3304, -5.1305,
          4.2383, -1.3442, -2.2525,  2.1067, -1.7842,  1.2646, -3.0602,  3.6656,
          3.4395, -3.7456,  0.1927, -1.2394, -0.6272,  4.1061,  2.1208,  1.4013,
         -0.7162,  2.6690,  4.2041,  0.9577,  0.1324, -0.4689],
        [-0.1706,  1.7912,  0.5456,  1.7628,  3.4748, -3.9629, -1.0447,  1.9246,
          0.7090,  1.9309,  2.3952,  0.3743, -0.2223, -1.3338,  0.0086, -0.5873,
         -4.6073, -5.3950, -1.3491,  4.1690, -0.1566, -1.3966,  1.9625, -0.1052,
         -0.8111, -1.3887, -0.0367, -3.6539,  0.0858,  4.8997],
        [-1.8056,  1.4997, -3.0894,  1.1877, -4.5314, -2.7503,  1.0923,  1.6539,
         -3.6214,  3.7660,  3.2089, -2.4037, -3.6043, -0.9404,  3.4390,  5.7618,
         -3.7532,  2.7549,  2.2914, -3.9960,  3.5746,  0.9593, -0.1924,  3.7767,
          0.6595,  4.7206, -1.9128,  4.6993, -2.0423, -3.3156],
        [ 3.7976, -0.8936,  3.7425,  1.1905,  3.730

In [26]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('fc1.weight', tensor([[-0.9455,  0.5285, -1.4916, -2.2217,  1.2970,  0.0291, -1.9377, -4.0828,
          3.1194, -1.5672, -1.4372,  1.9509, -1.5382,  0.9456, -2.3179,  3.6968,
          2.5201, -3.6483,  0.5055, -1.4644,  0.6884,  3.1010,  1.8234,  2.0452,
         -0.1579,  3.2199,  3.3649,  1.4777, -0.4170, -0.1961],
        [ 0.4975,  1.3478,  0.9962,  1.1588,  3.1973, -2.9054, -0.1706,  0.7057,
          1.0432,  1.1523,  1.7066,  0.4872,  0.3088, -1.4239,  0.0308, -0.8341,
         -3.2605, -4.2415, -1.5243,  3.4876, -0.2100, -1.2737,  1.7506, -0.3878,
         -0.7801, -1.5753, -0.4739, -3.6175,  0.3479,  4.2877],
        [-1.2877,  1.1549, -2.5278,  1.8092, -4.0246, -2.3739,  0.3292,  2.0196,
         -2.9842,  3.2538,  2.3749, -2.1960, -2.8251, -0.3124,  2.9302,  4.5777,
         -2.7264,  2.0390,  1.8290, -3.2232,  2.4892,  1.0321, -1.0419,  3.1020,
          0.7236,  3.8897, -1.3526,  3.9837, -1.8710, -2.5830],
        [ 2.4187, -0.8620,  2.3594,  0.6207,  2.975

## 对模型参数进行平均

In [27]:
from collections import OrderedDict

In [28]:
avg_updates = OrderedDict()
avg_updates["fc1.weight"] = (
    remote_model1_updates["fc1.weight"] + remote_model2_updates["fc1.weight"]
) / 2
avg_updates["fc1.bias"] = (
    remote_model1_updates["fc1.bias"] + remote_model2_updates["fc1.bias"]
) / 2
avg_updates["fc2.weight"] = (
    remote_model1_updates["fc2.weight"] + remote_model2_updates["fc2.weight"]
) / 2
avg_updates["fc2.bias"] = (
    remote_model1_updates["fc2.bias"] + remote_model2_updates["fc2.bias"]
) / 2

print(avg_updates)

OrderedDict([('fc1.weight', tensor([[-0.3738,  0.8023, -1.0190, -2.4035,  1.8510, -0.3301, -1.6341, -4.6066,
          3.6789, -1.4557, -1.8449,  2.0288, -1.6612,  1.1051, -2.6890,  3.6812,
          2.9798, -3.6970,  0.3491, -1.3519,  0.0306,  3.6035,  1.9721,  1.7233,
         -0.4370,  2.9445,  3.7845,  1.2177, -0.1423, -0.3325],
        [ 0.1635,  1.5695,  0.7709,  1.4608,  3.3360, -3.4342, -0.6076,  1.3151,
          0.8761,  1.5416,  2.0509,  0.4307,  0.0432, -1.3788,  0.0197, -0.7107,
         -3.9339, -4.8182, -1.4367,  3.8283, -0.1833, -1.3352,  1.8566, -0.2465,
         -0.7956, -1.4820, -0.2553, -3.6357,  0.2169,  4.5937],
        [-1.5467,  1.3273, -2.8086,  1.4984, -4.2780, -2.5621,  0.7108,  1.8368,
         -3.3028,  3.5099,  2.7919, -2.2999, -3.2147, -0.6264,  3.1846,  5.1697,
         -3.2398,  2.3969,  2.0602, -3.6096,  3.0319,  0.9957, -0.6172,  3.4393,
          0.6916,  4.3052, -1.6327,  4.3415, -1.9567, -2.9493],
        [ 3.1082, -0.8778,  3.0509,  0.9056,  3.352

## 更新模型参数

In [29]:
combined_model = SyNet(torch)

In [30]:
combined_model.load_state_dict(avg_updates)

In [31]:
del avg_updates, remote_model1_updates, remote_model2_updates

## 模型评估

In [34]:
#加载测试集
import pandas as pd
import numpy as np
datasets = pd.read_csv('./breast_eval.csv')
X_train = datasets.loc[:, datasets.columns != "y"]
X_train=X_train.loc[:,X_train.columns!="idx"]
y_train = datasets["y"]

test_data=torch.tensor(np.array(X_train),dtype=torch.float32)
test_target=torch.tensor(np.array(y_train),dtype=torch.float32)
test_target= test_target.view(test_target.shape[0],1)

## 测试初始模型

In [39]:
from sklearn.metrics import roc_curve, roc_auc_score
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = local_model(sample)
#         print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)
    a=torch.tensor([1.],dtype=torch.float32)
    b=torch.tensor([0.],dtype=torch.float32)
    y_hat=torch.where(y_hat>0.5,a,b)
    print(f"预测准确率: {(torch.sum(y_hat == test_target)/torch.tensor(test_target.size()[0],dtype=torch.float32)) * 100}%")

预测准确率: 39.0%


## 测试更新的全局模型

In [40]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)
    a=torch.tensor([1.],dtype=torch.float32)
    b=torch.tensor([0.],dtype=torch.float32)
    y_hat=torch.where(y_hat>0.5,a,b)
    print(f"预测准确率: {(torch.sum(y_hat == test_target)/torch.tensor(test_target.size()[0],dtype=torch.float32)) *100}%") 

Prediction: 0.9608038067817688 Ground Truth: 0.0
Prediction: 0.22233587503433228 Ground Truth: 0.0
Prediction: 0.7140563130378723 Ground Truth: 1.0
Prediction: 0.20813186466693878 Ground Truth: 0.0
Prediction: 0.574439287185669 Ground Truth: 1.0
Prediction: 0.29305562376976013 Ground Truth: 0.0
Prediction: 0.7404622435569763 Ground Truth: 1.0
Prediction: 0.49986031651496887 Ground Truth: 0.0
Prediction: 0.17904017865657806 Ground Truth: 1.0
Prediction: 0.9131640195846558 Ground Truth: 1.0
Prediction: 0.0064794691279530525 Ground Truth: 0.0
Prediction: 0.4520314037799835 Ground Truth: 0.0
Prediction: 0.2692161500453949 Ground Truth: 0.0
Prediction: 0.4681169390678406 Ground Truth: 1.0
Prediction: 0.23646485805511475 Ground Truth: 0.0
Prediction: 0.14534378051757812 Ground Truth: 1.0
Prediction: 0.7476869821548462 Ground Truth: 0.0
Prediction: 0.8671352863311768 Ground Truth: 0.0
Prediction: 0.7654181718826294 Ground Truth: 1.0
Prediction: 0.9981784820556641 Ground Truth: 0.0
Prediction: